# 6.S186 Hands-on Tutorial Walk-Through 

In [1]:
!pip install dexhub-api --upgrade
!pip install tensorflow
# Graphics and plotting.
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy

In [2]:
# set your API_KEY here
%env DEXHUB_API_TOKEN=32ypM1giGXZ9FfS6dYFrJbLD2z-dvRdfwNhOGW5Xr3U=

env: DEXHUB_API_TOKEN=32ypM1giGXZ9FfS6dYFrJbLD2z-dvRdfwNhOGW5Xr3U=


In [3]:
import dexhub
import os 
import mujoco
from mrl_utils.dataset_conversion import get_sim_and_dataset

/Users/yhpark/anaconda3/envs/mrl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step 1. Load Dataset from DexHub

In [4]:
# Specify the dataset UUID here 

dataset_uuid = "eef4a4db-1359-4230-bc38-a85254525292"

In [5]:
mjmodel, dataset = get_sim_and_dataset(dataset_uuid, dataset_name="younghyopark/random")

Dataset eef4a4db-1359-4230-bc38-a85254525292 already downloaded and extracted to ./dataset_eef4a4db-1359-4230-bc38-a85254525292.
Skipping video decoding for simulation data.
Skipping video decoding for simulation data.
Skipping video decoding for simulation data.


Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 260.68ba/s]


Skipping video decoding for simulation data.


Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 798.00ba/s]


Skipping video decoding for simulation data.


Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 712.23ba/s]


Skipping video decoding for simulation data.


Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 623.78ba/s]
Generating train split: 1337 examples [00:00, 93021.22 examples/s]
Svt[info]: -------------------------------------------
Svt[info]: SVT [version]:	SVT-AV1 Encoder Lib v2.2.0
Svt[info]: SVT [build]  :	Apple LLVM 15.0.0 (clang-1500.3.9.4)	 64 bit
Svt[info]: LIB Build date: Aug 22 2024 07:26:48
Svt[info]: -------------------------------------------
Svt[info]: Number of logical cores available: 12
Svt[info]: Number of PPCS 140
Svt[info]: [asm level on system : up to neon_i8mm]
Svt[info]: [asm level selected : up to neon_i8mm]
Svt[info]: -------------------------------------------
Svt[info]: SVT [config]: main profile	tier (auto)	level (auto)
Svt[info]: SVT [config]: width / height / fps numerator / fps denominator 		: 320 / 240 / 30 / 1
Svt[info]: SVT [config]: bit-depth / color format 					: 8 / YUV420
Svt[info]: SVT [config]: preset / tune / pred struct 					: 10 / PSNR / random access
Svt[info]: SVT [con

In [ ]:
# Visualize the dataset 


## Step 2. Create a Policy Network 

## Step 3. Train a Policy 

## Step 4. Evaluate your Policy!